In [1]:
#!/usr/bin/env python
# coding: utf-8
#===============================================================================
#
#           FILE: PPMI_ntb.py 
#         AUTHOR: Bianca Ciobanica
#	       EMAIL: bianca.ciobanica@student.uclouvain.be
#
#           BUGS: 
#        VERSION: 3.11.4
#        CREATED: 13-11-2023 
#
#===============================================================================
#    DESCRIPTION:  sources used : 
#                  https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html
#                  https://jofrhwld.github.io/teaching/courses/2022_lin517/lectures/word_vectors/02_vectors_examples.html
#                  https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.dok_matrix.html
#                  https://numpy.org/doc/stable/reference/generated/numpy.clip.html
#                  https://stackoverflow.com/questions/3391843/how-to-transform-negative-elements-to-zero-without-a-loop
#    
#          USAGE: 
#===============================================================================

In [2]:
import re
import time
from itertools import chain
import math
from nltk.corpus.reader import PlaintextCorpusReader
from nltk.lm import Vocabulary
from collections import Counter
import numpy as np
import pandas as pd
import scipy.sparse as sp

In [3]:
start_time_program = time.time()

In [4]:
corpus = PlaintextCorpusReader(root=".", 
                               fileids=["corpus.txt"])

In [5]:
#print(len(corpus.raw()))

In [6]:
def preprocess_steps(corpus):
    # get words
    text = corpus.sents()
    
    processed_text = [
        [token.lower() for token in re.sub(r"[.,:;!?\-\'\"\(\)\[\]]+", ' ', " ".join(sentence)).split() if token != ""]
        for sentence in text
    ]

    words = list(chain.from_iterable(processed_text))
    return words
    
preprocessed_words = preprocess_steps(corpus)

In [7]:
# ~~~~~ Create restricted voc ~~~~~
unk_cutoff = 10
#freqDist = Counter(sorted(words)) # sort alphabetically beforehand

#top_5_words = [item[0] for item in freqDist.most_common()[:5]] # get top 5
#print("\n".join(top_5_words))

In [8]:
def create_restricted_voc(words, threshold=None):
    word_counts = Counter(words)
    
    restricted_voc = ['<UNK>' if word_counts[word] < threshold else word for word in words]
    
    unique_tokens = set(restricted_voc)
    
    return restricted_voc, unique_tokens

In [9]:
def create_index(corpus, unique_tokens):
    words_index = {word: [] for word in unique_tokens}

    for idx, word in enumerate(corpus):
        words_index[word].append(idx)
    
    return words_index

In [10]:
words, unique_tokens = create_restricted_voc(preprocessed_words, threshold=unk_cutoff) # oov words replaced with unk in words array
voc_index = {word: idx for idx,word in enumerate (unique_tokens)}
words_index = create_index(words, unique_tokens)

In [11]:
voc_size = len(unique_tokens)
print("unique tokens :",voc_size)

alpha = 0.0001

unique tokens : 15200


In [12]:
# ~~~~~ Contextual windows  ~~~~~

def window(i):
    if i == 0: # first word
        return [words[i+1], words[i+2]] 
    if len(words) - i == 2: # penultimate word
        return [words[i-2], words[i-1], words[i+1]]
    if len(words) - i == 1: # last word
        return [words[i-2], words[i-1]]
                
    return [words[i-2], words[i-1], words[i+1], words[i+2]]
    
#print(",".join(window(0)))
#print(",".join(window(words.index("person"))))
#print(",".join(window(len(words) - 2)))


In [13]:
def co_occurrence_matrix_sparse(target_words):
    start_time = time.time()
    matrix = {target_word: Counter() for target_word in target_words} 
    
    for word in target_words:
        # get co-occurence vector for target word
        for idx in target_words[word]:
            target_context_words = window(idx)
            
            # update counts
            matrix[word].update(target_context_words) # we feed the context words to the counter object
                
    matrix_data_dict = list(matrix.values()) # is a list of counters, so word => vector 

    # create a df from iterable dict
    occurences_matrix = pd.DataFrame(matrix_data_dict, index=list(target_words), columns=list(target_words))
    occurences_matrix = occurences_matrix.fillna(0).astype(int) # fill 0 for words not occuring with others

    end_time = time.time()
    execution_time = end_time - start_time
    
    print(f"Execution time: {execution_time} seconds")

    return occurences_matrix
    
#term_context_matrix_sparse = co_occurrence_matrix_sparse(words_index)

In [14]:
# P(w_i, c_j) = C(w_i, c_j) + alpha / sum( C(w_i,c_j) + alpha )
# P(w_i) = sum_c_j ( C(w_i, c_j )
# P(c_j) = sum_w_i ( C(w_i, c_j )
def create_ppmi_sparse(matrix, alpha):
    total_counts = sum((matrix + alpha).sum())

    mle_matrix =  (matrix + alpha) / total_counts
    p_wi = np.sum(matrix, axis=1)/ total_counts
    p_cj = np.sum(matrix, axis=0) / total_counts
    ppmi_sparse = np.log2(mle_matrix / ( p_wi  *  p_cj ))
    ppmi_sparse = ppmi_sparse.clip(0)
    
    return ppmi_sparse
    
#ppmi_sparse = create_ppmi_sparse(term_context_matrix_sparse, alpha)

In [15]:
def co_occurrence_matrix(target_words):
    start_time = time.time()
    
    matrix = {target_word: Counter() for target_word in target_words} 
    
    for word in target_words:
        # get co-occurence vector for target word
        for idx in target_words[word]: # position in original corpus
            target_context_words = window(idx)
            
            # update counts
            matrix[word].update(target_context_words) # we feed the context words to the counter object

    end_time = time.time()
    execution_time = end_time - start_time
    
    print(f"Execution time: {execution_time} seconds")
    return matrix
    
term_context_matrix = co_occurrence_matrix(words_index)

Execution time: 2.4832537174224854 seconds


In [16]:
def calculate_total_MLE(matrix):
    mle = 0
    for counter in matrix.values():
        mle += sum(count for count in counter.values())
            
    return mle

In [17]:
def calculate_p_cj(matrix):
    p_cj = 0
    context_counts = Counter()

    for word, context_counter in matrix.items():
        # add each key which is the context word and its count to the counter
        context_counts += context_counter 
    return context_counts

In [18]:
# P(w_i, c_j) = C(w_i, c_j) + alpha / sum( C(w_i,c_j) + alpha )
# P(w_i) = sum_c_j ( C(w_i, c_j )
# P(c_j) = sum_w_i ( C(w_i, c_j )

def create_ppmi(matrix, alpha):
    start_time = time.time()
    
    total_context_counts = calculate_total_MLE(matrix) + (alpha * len(matrix))
    print(total_context_counts)
    
    context_counts = calculate_p_cj(matrix)
    # or mle = sum(count for counter in term_context_matrix.values() for count in counter.values()) + ( alpha * (voc_size **2))

    for word in matrix:
        p_wi = sum(matrix[word].values()) / total_context_counts # w_i frequency
        
        for context_w, count in matrix[word].items():
            p_wi_cj = (count + alpha) / total_context_counts # p(context_j co-occuring with target w_i)
            p_cj = context_counts[context_w] / total_context_counts
            ppmi  = math.log2(p_wi_cj / (p_wi * p_cj))
            matrix[word][context_w] = max(ppmi, 0)

    end_time = time.time()
    execution_time = end_time - start_time
    
    print(f"Execution time: {execution_time} seconds")
    return matrix

ppmi_matrix = create_ppmi(term_context_matrix, alpha)

9816888.52
Execution time: 10.765803813934326 seconds


In [19]:
def get_cosine_sim(word1, word2):
    """ input : two vectors of two words
        return : cosine similarity value
    """
    vector1 = ppmi_matrix[word1]
    vector2 = ppmi_matrix[word2]

    dot_product = 0

    common_words = set(vector1.keys()) & set(vector2.keys())
    for word in common_words:
        dot_product += vector1[word]  * vector2[word]

    v1_norm = np.linalg.norm(list(c for c in vector1.values()))
    v2_norm = np.linalg.norm(list(c for c in vector2.values()))
    # or without numpy
    #v1_norm = math.sqrt(sum(map(lambda x: x*x, vector1.values())))
    #v2_norm = math.sqrt(sum(map(lambda x: x*x, vector2.values())))
    total_counts = v1_norm * v2_norm

    normalized_dotprod = dot_product / total_counts
    
    return normalized_dotprod

In [20]:
def get_cosine_sparse(word1,word2):

    vector1 = ppmi_sparse.loc[word1]
    vector2 = ppmi_sparse.loc[word2]
    
    
    dot_product = np.inner(vector1, vector2)

    v1_norm = np.linalg.norm(vector1)
    v2_norm = np.linalg.norm(vector2)

    
    # or without numpy
    #v1_norm = math.sqrt(sum(map(lambda x: x*x, vector1)))
    #v2_norm = math.sqrt(sum(map(lambda x: x*x, vector2)))
    total_counts = v1_norm * v2_norm

    normalized_dotprod = dot_product / total_counts

    return normalized_dotprod

In [21]:
def get_5_closest_words(target):
    similarities = {}
    #similarities_sparse = {}
    
    for w in unique_tokens:
        if w == target:
            continue
        sim = get_cosine_sim(target,w)
        #sim_sparse = get_cosine_sparse(target,w)

        similarities[w] = sim
       # similarities_sparse[w] = sim_sparse
                                         
    top_5_words = sorted(similarities, key=similarities.get, reverse=True)[:5]
    #top_5_words_sparse = sorted(similarities, key=similarities.get, reverse=True)[:5]
    
    print("Top 5 words similar to " + target)
    for w in top_5_words:
        print(f"{w}: {similarities[w]}")
    #print("From scratch and no sparse".ljust(50), "Sparse matrix\n")
    #for w, w_sparse in zip(top_5_words, top_5_words_sparse):
     #   print(f"{w}: {similarities[w]}".ljust(50), 
    #        f"{w_sparse}: {similarities_sparse[w_sparse]}")
    print()
    
    return set(top_5_words)

In [22]:
car_closest = get_5_closest_words('car')
feature_closest = get_5_closest_words('feature')
computer_closest = get_5_closest_words('computer')

Top 5 words similar to car
cars: 0.17545540185356884
vehicles: 0.13381011499813825
electric: 0.13203808511780504
fuel: 0.1146915686314566
a: 0.11134307878896335

Top 5 words similar to feature
features: 0.12279135136721567
film: 0.10958183870864024
most: 0.10227191575051996
roles: 0.10030301352910866
films: 0.09936376474575513

Top 5 words similar to computer
graphics: 0.14414894689831118
software: 0.1384913207106095
technology: 0.13741492287690896
system: 0.13550690971258733
programming: 0.13514113451579493



In [23]:
print("5 closest words to car: \n", car_closest)
print("5 closest words to feature: \n",feature_closest)
print("5 closest words to computer: \n",computer_closest)

5 closest words to car: 
 {'electric', 'vehicles', 'fuel', 'a', 'cars'}
5 closest words to feature: 
 {'most', 'features', 'film', 'films', 'roles'}
5 closest words to computer: 
 {'technology', 'programming', 'graphics', 'software', 'system'}


In [24]:
end_time_program = time.time()
total_execution_time = end_time_program - start_time_program
    
print(f"Total execution time: {total_execution_time} seconds")

Total execution time: 20.51163363456726 seconds
